In [85]:
%%html
<style>
.prompt_container { display: none !important; }
.prompt { display: none !important; }
.run_this_cell { display: none !important; }

.slides {
    position: absolute;
    top: 0;
    left: 0;
}
</style>

# ProgRes, Part IIb

# Web servers

Fabien Mathieu - fabien.mathieu@normalesup.org

Sébastien Tixeuil - Sebastien.Tixeuil@lip6.fr

# Methodology

- Course, practicals, and mini-projects are made on notebooks (jupyter or jupyterlab)
- Available on Github: https://github.com/balouf/progres
- This means you will send your practical notebooks. 
- Please put your name(s) **on the filename AND inside** as well!

# The rules

https://github.com/balouf/progres/blob/main/rules.ipynb

# Roadmap

- Part I: done
- Part II (Web services)
  - Last week: Client side
  - This week: Server side
    - Bottle
    - Testing
    - Web Server + API
- Part III: P2P

## Create an API in Python

# Reminder

HTTP can be used to communicate:
- Between the server and the user (frontend)
- Internally, between the server and other services (backend)

API (Application Programming Interface) is a set of exposed methods for interaction between programms.

REST is a *code of conduct* for http-based API.

# Anatomy of a Web server

A web server can be made of multiple parts:

HTTP Server:
- Listen to incoming HTTP requests
- Dispatch them:
  - Fetch local files
  - Send to application

Application server:
- Process the requests
  - At frontend
  - At backend

# Web libraries

Many libraries / framework exist in Python.

From https://www.educative.io/blog/top-python-web-frameworks

1. Flask: light, yet powerful. Adapted to middle-complexity websites.
2. Django: ultra-powerful, ultra-complete. Adapted to production-ready complex websites.
3. FastAPI: light, robust, easy-to-learn, production-ready.
4. CherryPy: Flask simpler and more pythonic. Adapted to professors.
5. Bottle: very light, standalone. Adapted to courses and mini-projects.


# The bottle framework

- **Routing**: Requests to function-call mapping with support for clean and dynamic URLs.
- **Templates**: Fast and pythonic built-in template engine.
- **Utilities**: Convenient access to form data, file uploads, cookies, headers and other HTTP-related metadata.
- **Server**: 
  - Built-in HTTP development server (mono-thread, easy to block...)
  - Support for other WSGI capable HTTP server.

WSGI is the Web Server Gateway Interface (allows to combine multiple frameworks together).

# Hello world example

In [3]:
from bottle import route, run
@route('/hello/<name>')
def index(name):
    return 'Hello '+name+'!'
run(host='localhost', port=8080)

Bottle v0.12.25 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [18/Oct/2024 14:03:32] "GET /hello/World HTTP/1.1" 200 12
127.0.0.1 - - [18/Oct/2024 14:03:45] "GET /hello/students HTTP/1.1" 200 15


Check that the server works: http://localhost:8080/hello/World

# Hello world example explained

- `route` is a Python decorator.
  - Decorators are used to modify the behavior of a method
  - Here `@route` links the method `index(name)` to the route `/hello/<name>`
- `run` starts the Web server
  - ``host=`localhost` ``: listen local incoming connections
  - `port`: listening port

# Intermezzo: Jupyter/IPython magics 

- IPython Magics are powerful commands that change the behavior of a code cell.
- Here we will use some to avoid switching back and forth between the notebook and command line.
- Using magics is never mandatory but often helpful
- If you want to learn more about magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html

# Hello world example with magics

In [5]:
%%writefile run.py
from bottle import route, run
@route('/hello/<name>')
def index(name):
    return f'Hello {name}!'
run(host='localhost', port=8080)

Overwriting run.py


In [6]:
!wt python run.py

Check that the server works: http://localhost:8080/hello/Students

# Hello world example a bit cleaner

In [8]:
%%writefile app.py
from bottle import Bottle
from html import escape
from gevent import monkey
monkey.patch_all()
app = Bottle()
@app.route('/')
def alive():
    start = ["Server running. Available routes:<br><ul>"]
    return start+[f"<li><a href='{r.rule}'>{escape(r.rule)}</a></li>" 
                  for r in app.routes]+['</ul>']

Overwriting app.py


In [9]:
%%writefile run.py
from app import app
@app.route('/hello/<name>')
def index(name):
    return f'Hello {name}!'
app.run(server='gevent')

Overwriting run.py


In [10]:
!wt python run.py

http://localhost:8080

# Filters: :int, :float, :path, :re

In [11]:
%%writefile run.py
from app import app
from bottle import static_file
@app.route('/square/<i:int>')
def square(i):
    return f'The square of {i} is {i**2}.'
@app.route('/open/<filepath:path>')
def open_file(filepath):
    return static_file(filepath, root='.')
@app.route('/slug/<id_slug:re:[0-9]+-.*>')
def slug(id_slug):
    id_slug = id_slug.split('-', 1)
    return f'Slug {id_slug[1]} has id {id_slug[0]}.'
app.run(server='gevent')

Overwriting run.py


In [12]:
!wt python run.py

http://localhost:8080

# Specify the method (GET, POST, ...)

In [13]:
%%writefile run.py
from app import app
from bottle import static_file
@app.route('/square/<i:int>', method='get')
def square(i):
    return f'The square of {i} is {i**2}.'
@app.get('/open/<filepath:path>') # Equivalent
def open_file(filepath):
    return static_file(filepath, root='.')
@app.get('/slug/<id_slug:re:[0-9]+-.*>')
def slug(id_slug):
    id_slug = id_slug.split('-', 1)
    return f'Slug {id_slug[1]} has id {id_slug[0]}.'
app.run(server='gevent')

Overwriting run.py


In [14]:
!wt python run.py

http://localhost:8080

# A route can return...

In [15]:
%%writefile run.py
from app import app
import json
from bottle import static_file
@app.get('/nothing')
def nothing():
    return None
@app.get('/string')
def string():
    return json.dumps({'a': 42, 'b': None})
@app.get('/file')
def file():
    return static_file('run.py', root='.')
@app.get('/iterator')
def iterator():
    return ['Line 1\n', 'Line 2 \n']
app.run(server='gevent')

Overwriting run.py


In [16]:
!wt python run.py

http://localhost:8080

# The response object

In [21]:
%%writefile run.py
from app import app
import json
from bottle import response
@app.route('/powers/<i:int>')
def powers_json(i):
    response.content_type = 'application/json; charset=utf-8'
    return json.dumps({'i':i, 'square': i**2, 'cube': i**3})
@app.route('/powers_txt/<i:int>')
def powers_txt(i):
    return json.dumps({'i':i, 'square': i**2, 'cube': i**3})
app.run(server='gevent')

Overwriting run.py


In [20]:
!wt python run.py

http://localhost:8080

- When returning a string, it is encoded with respect to the given content-type (only bytes are sent back to the client)
- Here, the server sends a byte array encoded in utf-8, that represents an object formatted in JSON.

# Sending Errors, Redirects

In [22]:
%%writefile run.py
from app import app
from bottle import abort, redirect, static_file
@app.route('/private/<id:int>')
def private(id):
    abort(401, "Sorry, access denied.")
@app.route('/open/<filepath:path>')
def open_file(filepath):
    return static_file(filepath, root='.')
@app.route('/json/<stem>')
def getjson(stem):
    redirect(f"/open/{stem}.json")
app.run(server='gevent')

Overwriting run.py


In [23]:
!wt python run.py

http://localhost:8080

## Testing your code

# Why testing is important?

- Domino error
  - A function `two` returns 3 instead of 2
  - In a far, far away submodule, lies `4**two()`
- Coverage:
  - Are you sure you don't have dead code lying?
  - https://balouf.github.io/stochastic_matching/readme.html
- Quick reaction:
  - When you break things
  - When another dev breaks things
  - When the new release of an obscure package breaks things 

# How to test?

- unittest
  - The original testing framework
  - You should not use it (too heavy!)
- pytest
  - like unittest but simpler
  - You should use it!
- doctests (not covered here but nice!)
  
 https://www.lincs.fr/events/testing-in-python/

# pytest syntax

If it starts with `test`, it's a test.

In [18]:
!del test_*.py

Impossible de trouver C:\Users\fabienma\git\courses\progres\test_*.py


In [28]:
%%writefile test_file.py

def square(x):
    return x**2

def test_square():
    assert square(2)==4

Overwriting test_file.py


In [29]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.11.4, pytest-7.4.2, pluggy-1.3.0
rootdir: C:\Users\fabienma\git\courses\progres
plugins: anyio-4.3.0, Faker-30.3.0
collected 1 item

test_file.py .                                                           [100%]

============================== 1 passed in 0.14s ==============================


# Example: arithmetic testing

In [37]:
%%writefile run_maths.py
from app import app
from json import dumps
@app.route("/add/<i:int>/<j:int>")
def add(i,j):
    return dumps([i+j])
@app.route("/sub/<i:int>/<j:int>")
def sub(i,j):
    return dumps([i-j])

@app.route("/mul/<i:int>/<j:int>")
def mul(i,j):
    return dumps([i*j])
@app.route("/div/<i:int>/<j:int>")
def div(i,j):
    return dumps([i//j,i%j])
app.run(host='localhost', port=8070, server='gevent')

Overwriting run_maths.py


In [38]:
!wt python run_maths.py

http://localhost:8070

# Example: arithmetic testing

In [32]:
from requests import get
server_ip = "127.0.0.1"
server_port = 8070
r1 = get(f"http://{server_ip}:{server_port}/add/4/5")
print(r1.text)
r2 = get(f"http://{server_ip}:{server_port}/add/7/14")
print(r2.text)

[9]
[21]


# Example: arithmetic testing

In [33]:
%%writefile test_maths.py
from requests import get
from json import loads
server_ip = "127.0.0.1"
server_port = 8070
def test_add():
    r1 = get(f"http://{server_ip}:{server_port}/add/4/5")
    assert loads(r1.text) == [ 9 ]
    r2 = get(f"http://{server_ip}:{server_port}/add/7/14")
    assert loads(r2.text) == [21]

Writing test_maths.py


In [34]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.11.4, pytest-7.4.2, pluggy-1.3.0
rootdir: C:\Users\fabienma\git\courses\progres
plugins: anyio-4.3.0, Faker-30.3.0
collected 2 items

test_file.py .                                                           [ 50%]
test_maths.py .                                                          [100%]

============================== 2 passed in 0.38s ==============================


# Example: arithmetic testing

In [39]:
%%writefile test_maths.py
from requests import get
from json import loads
server_ip = "127.0.0.1"
server_port = 8070
def test_add():
    r1 = get(f"http://{server_ip}:{server_port}/add/4/5")
    assert loads(r1.text) == [ 9 ]
    r2 = get(f"http://{server_ip}:{server_port}/add/7/14")
    assert loads(r2.text) == [21]
def test_sub():
    r1 = get(f"http://{server_ip}:{server_port}/sub/4/5")
    assert loads(r1.text) == [ -1 ]
    r2 = get(f"http://{server_ip}:{server_port}/sub/2/2")
    assert loads(r2.text) == [0]

Overwriting test_maths.py


In [40]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.11.4, pytest-7.4.2, pluggy-1.3.0
rootdir: C:\Users\fabienma\git\courses\progres
plugins: anyio-4.3.0, Faker-30.3.0
collected 3 items

test_file.py .                                                           [ 33%]
test_maths.py ..                                                         [100%]

============================== 3 passed in 0.34s ==============================


# Example: strings

In [41]:
%%writefile run_string.py
from app import app
@app.route("/to_upper/<s>")
def to_upper(s):
    return s.upper()
app.run(host='localhost', port=8060, server='gevent')

Writing run_string.py


In [42]:
!wt python run_string.py

http://localhost:8060

# Example: strings

In [43]:
%%writefile test_strings.py
from requests import get
server_ip = "127.0.0.1"
server_port = 8060
def test_upper():
    r1 = get(f"http://{server_ip}:{server_port}/to_upper/LaTeX")
    assert r1.text == 'LATEX'
    r2 = get(f"http://{server_ip}:{server_port}/to_upper/Été")
    assert r2.text == 'ÉTÉ'

Writing test_strings.py


In [44]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.11.4, pytest-7.4.2, pluggy-1.3.0
rootdir: C:\Users\fabienma\git\courses\progres
plugins: anyio-4.3.0, Faker-30.3.0
collected 4 items

test_file.py .                                                           [ 25%]
test_maths.py ..                                                         [ 75%]
test_strings.py .                                                        [100%]

============================== 4 passed in 0.39s ==============================


## API Hour


# A First micro-service example

In [45]:
%%writefile run.py
ip="127.0.0.1"
math_port=8070
from app import app
from bottle import request
from requests import get
from json import loads
form = """<form action="/" method="post">
 i: <input name="i" type="text" />
 j: <input name="j" type="text" />
<input value="Add" type="submit" />
</form>"""
@app.get("/")
def input_form():
    return form
@app.post("/")
def process_form():
    i = request.forms['i']
    j = request.forms['j']
    r = get(f"http://{ip}:{math_port}/add/{i}/{j}")
    l = loads(r.text)
    return f"<h1>{i}+{j}={l[0]}</h1>"    
app.run(server='gevent')

Overwriting run.py


In [46]:
!wt python run.py

http://localhost:8080

# What happened?

- Client (browser) requests a get method to 8080
- 8080 responds to get a html form to client
- Client requests a post method to 8080
- 8080 requests a get method to 8070
- 8070 responds to get a json to 8080
- 8080 responds to post a html result to client

# Another one?

In [47]:
%%writefile run.py
ip="127.0.0.1"
string_port=8060
from app import app
from bottle import request
from requests import get
form = """<form action="/" method="post">
 Word: <input name="s" type="text" />
 <input value="To Uppercase" type="submit" />
 </form>"""
@app.get("/")
def input_form():
    return form
@app.post("/")
def process_form():
    s = request.forms['s'] # unicode is universal?
    r = get(f"http://{ip}:{string_port}/to_upper/{s}")
    l = r.text
    return f"<h1>{s}.upper()={l}</h1>"    
app.run(server='gevent')

Overwriting run.py


In [48]:
!wt python run.py

http://localhost:8080

# Getting parameters:

- from the URL, e.g. `/entry?order=name`: `request.query['order']`
- from the post data, e.g. a posted form: `request.forms['order']`
- from URL or post data: `request.params['order']`
- `request.params.order`: variant with auto-encoding management
- Retrieve a posted file:
  - `image = request.files['image']`
  - `image.save('images/uploaded')`

# A bit of practice

- Here we used different port numbers to demonstrate interactions between several machine
- In practice, if the machines are distinct, you use the same port
- Multiple services can run on the same machine
  - One developper writes the maths routes
  - One developper writes the form routes
  - One developper writes the server
- Good practice: don't use http for internal calls (inside the same service)
- But sometimes, it's better than running X servers

# First example revisited: the backend

In [49]:
%%writefile app_maths.py
from json import dumps
from app import app
@app.route("/add/<i:int>/<j:int>")
def add(i,j):
    return dumps([i+j])
@app.route("/sub/<i:int>/<j:int>")
def sub(i,j):
    return dumps([i-j])
@app.route("/mul/<i:int>/<j:int>")
def mul(i,j):
    return dumps([i*j])
@app.route("/div/<i:int>/<j:int>")
def div(i,j):
    return dumps([i//j,i%j])

Writing app_maths.py


In [50]:
%%writefile app_strings.py
from app import app
@app.route("/to_upper/<s>")
def to_upper(s):
    return s.upper()

Writing app_strings.py


# First example revisited: the frontend

In [52]:
%%writefile app_user.py
from bottle import Bottle, request
from json import loads
import requests
app = Bottle()
@app.get("/")
def input_form():
    return """<form action="/" method="post"> i: <input name="i" type="text" /> j: <input name="j" type="text" />
<input type="hidden" name="action" value="add" /> <input value="Add" type="submit" /></form>
<br>
<form action="/" method="post"> Word: <input name="s" type="text" /> <input value="To Uppercase" type="submit" />
 <input type="hidden" name="action" value="up" /> </form>"""
@app.post("/")
def process_form():
    action = request.params['action']
    if action == 'add':
        i = request.params['i']
        j = request.params['j']
        r = requests.get(f"http://127.0.0.1:8080/maths/add/{i}/{j}")
        l = loads(r.text)
        return f"<h1>{i}+{j}={l[0]}</h1>"
    else:
        r = requests.get(f"http://127.0.0.1:8080/strings/to_upper/{request.forms['s']}")
        l = r.text
        return f'<h1>"{request.forms.s}".upper()={l}</h1>'    

Overwriting app_user.py


# First example revisited: the server

In [53]:
%%writefile run.py
from gevent import monkey; monkey.patch_all()
from bottle import Bottle
from app_user import app as app_user
from app_maths import app as app_maths
from app_strings import app as app_strings
app = Bottle()
app.mount('/maths', app_maths)
app.mount('/strings', app_strings)
app.merge(app_user)
app.run(server='gevent')

Overwriting run.py


In [54]:
!wt python run.py

http://localhost:8080

## Templates

# Templates

In [56]:
x=2
"x=%s" % x

'x=2'

- Writing nice html requires lot of boring text (tags, etc...)
- With templates, you can prepare skeletons with placeholders to save times
- Many possibilities:
  - Native Python: f-strings, % substitution, string.Template
  - Some frameworks offer very complex templates (e.g. [Jinja](https://jinja.palletsprojects.com/en/latest/templates))
  - bottle provides its own template factory

# Basic templates

In [60]:
from bottle import SimpleTemplate, template
tpl = SimpleTemplate('Hello {{name}}!')
s = tpl.render(name='World')
print(s)
my_dict={'number': '123', 'street': 'Fake St.', 'city': 'Fakeville'}
s = template('The address is at {{number}} {{street}}, {{city}}', my_dict)
print(s)

Hello World!
The address is at 123 Fake St., Fakeville


# If condition

In [61]:
tpl = 'Hello {{name if name != "World" else "Planet"}}!'
s = template(tpl, name='World')
print(s)
s = template(tpl, name='Sébastien')
print(s)

Hello Planet!
Hello Sébastien!


# Warning: HTML entities

In [63]:
from IPython.display import HTML
tpl = 'Hello {{name}}!'
s = template(tpl, name='<b>World</b>')
HTML(s)

In [64]:
tpl = 'Hello {{!name}}!'
s = template(tpl, name='<b>World</b>')
HTML(s)

# Template files

In [65]:
%%writefile simple.tpl
<html>
 <head><title>{{title}}</title></head>
 <body>
 <h1>{{title}}</h1>
 {{content}}
 </body>
</html

Writing simple.tpl


# Template files

In [66]:
 site = { 'title': 'This is the title', 
'content': 'This is the content'}
s = template('simple.tpl',site)
print(s)
HTML(s)

<html>
 <head><title>This is the title</title></head>
 <body>
 <h1>This is the title</h1>
 This is the content
 </body>
</html



# Conditions in template files

In [67]:
%%writefile simple_if.tpl
<html>
 <head><title>{{title}}</title></head>
 <body>
 <h1>{{title}}</h1>
 % if bold_content:
 <b>
 % end
 {{content}}
 % if bold_content:
 </b>
 % end
 </body>
</html>

Writing simple_if.tpl


In [70]:
site = { 'title': 'This is the title', 
'content': 'This is the content'}
s = template('simple_if.tpl',site, bold_content=False)
HTML(s)

# Template files loops

In [71]:
%%writefile simple_for.tpl
<html>
 <head><title>{{title}}</title></head>
 <body>
 <h1>{{title}}</h1><ul>
 % for content in contents:
 % if bold:
 <li><b>{{content}}</b></li>
 % else:
 <li>{{content}}</li>
 % end
 %end
 </ul></body>
</html>

Writing simple_for.tpl


In [74]:
site = { 'title': 'This is the title', 'contents': 
        ['This is the first line of content', 'second line', 'third line']}
s = template('simple_for.tpl', site, bold=False)
HTML(s)

# The end!